In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Setting random seed for reproducibility
torch.manual_seed(140)
np.random.seed(140)
random.seed(140)

In [ ]:

"""
# Load data
x_values = torch.tensor([1, 2, 3]) # X values
y_values = torch.tensor([20, 30, 40]) # Y values

x_values_np = x_values.numpy()
y_values_np = y_values.numpy()

# Plot data
plt.plot(x_values_np, y_values_np)
plt.xlabel('Time')
plt.ylabel('Values')
plt.title('Values over time')
plt.show()

"""


In [ ]:
# Simulate data 
x_values = torch.tensor([1, 2, 3], dtype=torch.float32).view(-1, 1, 1) # X values
y_values = x_values * 2 # Y values

# Test data
#x_test = torch.tensor([1, 2, 3, 4, 5, 6], dtype=torch.float32).view(-1, 1, 1) # X values
#y_test = x_test * 10 # Y values

print(x_values)


sc_x = MinMaxScaler()
sc_y = MinMaxScaler()

# Shape the data from 3D to 2D
x_values_2D = x_values.squeeze(-1).numpy()
y_values_2D = y_values.squeeze(-1).numpy()

# Fit the scaler and transform the data
x_scaled = sc_x.fit_transform(x_values_2D)
y_scaled = sc_y.fit_transform(y_values_2D)

# Reshape the data back to 3D and convert to tensor
x_scaled = torch.tensor(x_scaled).unsqueeze(-1)
y_scaled = torch.tensor(y_scaled).unsqueeze(-1)

# Print shapes
print(x_scaled)
print(y_scaled.shape)

In [ ]:
# Define the LSTM model
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.num_layers = num_layers


        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        x, _ = self.lstm(x, (h0, c0))
        #x, _ = self.lstm(x)
        # Extract only the last timestep's output for prediction
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [ ]:
num_epochs = 50
learning_rate = 0.05

input_size = 1
hidden_size = 10
num_layers = 1

num_classes = 1

# Initialize the model
model = SimpleLSTM(input_size, hidden_size, num_classes, num_layers)
#model = SimpleGRU()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# Train the model
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(x_scaled)
    loss = loss_fn(output, y_scaled.view(-1, 1))
    loss.backward()
    optimizer.step()
    
    if epoch % 5 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item()}')

In [ ]:
# Assuming `sc_x` and `sc_y` are already fitted to your training data
# Prepare multiple x values for prediction
x_to_predict = torch.tensor([1, 2, 3, 4], dtype=torch.float32).view(-1, 1, 1)  # Example x values
x_to_predict_2D = x_to_predict.squeeze(-1).numpy()  # Convert to 2D for scaling
x_to_predict_scaled = sc_x.transform(x_to_predict_2D)  # Scale x values
x_to_predict_scaled_tensor = torch.tensor(x_to_predict_scaled, dtype=torch.float32).view(-1, 1, 1)  # Convert back to tensor

model.eval()
predictions_scaled = []
with torch.no_grad():
    for x_val in x_to_predict_scaled_tensor:
        prediction_scaled = model(x_val.unsqueeze(0))  # Add batch dimension
        predictions_scaled.append(prediction_scaled)

# Convert predictions to a numpy array for inverse transform
predictions_scaled_np = torch.cat(predictions_scaled, dim=0).numpy()
predictions = sc_y.inverse_transform(predictions_scaled_np.reshape(-1, 1))  # Inverse transform predictions

# Print Actual vs Predicted
actual_y = (x_to_predict * 2).numpy().flatten()  # Replace with actual calculation or data as needed
predicted_y = predictions.flatten()

print("X Value\tActual Y\tPredicted Y")
for i, x_val in enumerate(x_to_predict.numpy().flatten()):
    print(f"{x_val}\t{actual_y[i]}\t{predicted_y[i]}")

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(x_to_predict.numpy().flatten(), actual_y, 'bo-', label='Actual')
plt.plot(x_to_predict.numpy().flatten(), predicted_y, 'ro-', label='Predicted')
plt.title('LSTM Predictions vs Actual')
plt.xlabel('X Value')
plt.ylabel('Y Value')
plt.legend()
plt.show()
